In [3]:
###########
# IMPORTS #
###########
%matplotlib inline

import cv2
import os
from matplotlib import pyplot as plt

In [4]:
###############
# FRAME SPLIT #
###############

curr_dir = os.getcwd()
vid_clips_2020 = curr_dir + '/2020/VideoClips/'

def split_clip(directory):
    for filename in os.listdir(directory):
        if filename.endswith(".mp4"):
            video = directory + filename
            vidcap = cv2.VideoCapture(video)
            success,image = vidcap.read()
            count = 0
            new_dir = curr_dir + '/preprocessed/' + os.path.splitext(filename)[0] + '/'
            while success:
                frame_file = "%sframe%d.jpg" % (new_dir, count)
                if not os.path.exists(os.path.dirname(frame_file)):
                    try:
                        os.makedirs(os.path.dirname(frame_file))
                    except OSError as exc: # Guard against race condition
                        if exc.errno != errno.EEXIST:
                            raise
                
                cv2.imwrite(frame_file, image)     # save frame as JPEG file      
                success,image = vidcap.read()
                count += 1
            print('Split video', video)
        else:
            continue

split_clip(vid_clips_2020)

Split video /Users/vidyaakavoor/Documents/school/CS640/Project/cs640-project-fall2020/local_preprocessing/2020/VideoClips/klobuchar_2_0.mp4
Split video /Users/vidyaakavoor/Documents/school/CS640/Project/cs640-project-fall2020/local_preprocessing/2020/VideoClips/buttigieg_1_2.mp4
Split video /Users/vidyaakavoor/Documents/school/CS640/Project/cs640-project-fall2020/local_preprocessing/2020/VideoClips/warren_11_6.mp4
Split video /Users/vidyaakavoor/Documents/school/CS640/Project/cs640-project-fall2020/local_preprocessing/2020/VideoClips/sanders_3_0.mp4
Split video /Users/vidyaakavoor/Documents/school/CS640/Project/cs640-project-fall2020/local_preprocessing/2020/VideoClips/yang_2_3.mp4
Split video /Users/vidyaakavoor/Documents/school/CS640/Project/cs640-project-fall2020/local_preprocessing/2020/VideoClips/biden_21_3.mp4


In [5]:
##################
# FACE DETECTION #
##################

face_coordinates = {}
eye_coordinates = {}
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eyes_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

def find_face(directory):
    for filename in os.listdir(directory):
        if filename.endswith(".mp4"):
            frame_dir = curr_dir + '/preprocessed/' + os.path.splitext(filename)[0] + '/'
            for frame in os.listdir(frame_dir):
                frame_name = frame_dir + frame
                img = cv2.imread(frame_name)
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

                faces_detected = face_cascade.detectMultiScale(img, scaleFactor=1.1, minNeighbors=5)
                if len(faces_detected) > 0:
                    (x, y, w, h) = faces_detected[0]
                    face_coordinates[frame_name] = (x, y, w, h)
                    
#                     eyes = eyes_cascade.detectMultiScale(img[y:y+h, x:x+w])
#                     eye_coordinates[frame_name] = [(ex, ey, ew, eh) for (ex, ey, ew, eh) in eyes]
#                     for (ex, ey, ew, eh) in eyes:
#                         cv2.rectangle(img, (x+ex, y+ey), (x+ex+ew, y+ey+eh), (255, 255, 255), 1)
#                         plt.imshow(img)
#                         plt.show()

find_face(vid_clips_2020)
print(face_coordinates)

{'/Users/vidyaakavoor/Documents/school/CS640/Project/cs640-project-fall2020/local_preprocessing/preprocessed/klobuchar_2_0/frame187.jpg': (254, 98, 100, 100), '/Users/vidyaakavoor/Documents/school/CS640/Project/cs640-project-fall2020/local_preprocessing/preprocessed/klobuchar_2_0/frame85.jpg': (241, 115, 90, 90), '/Users/vidyaakavoor/Documents/school/CS640/Project/cs640-project-fall2020/local_preprocessing/preprocessed/klobuchar_2_0/frame91.jpg': (240, 113, 88, 88), '/Users/vidyaakavoor/Documents/school/CS640/Project/cs640-project-fall2020/local_preprocessing/preprocessed/klobuchar_2_0/frame150.jpg': (253, 105, 99, 99), '/Users/vidyaakavoor/Documents/school/CS640/Project/cs640-project-fall2020/local_preprocessing/preprocessed/klobuchar_2_0/frame144.jpg': (257, 101, 103, 103), '/Users/vidyaakavoor/Documents/school/CS640/Project/cs640-project-fall2020/local_preprocessing/preprocessed/klobuchar_2_0/frame46.jpg': (254, 131, 89, 89), '/Users/vidyaakavoor/Documents/school/CS640/Project/cs640

In [6]:
#################
# FACE CROPPING #
#################

def crop_faces(padding=5):
    for pic_name in face_coordinates:
        (x, y, w, h) = face_coordinates[pic_name]
        img = cv2.imread(pic_name)
        try:
            # NOTE: this throws an error for some reason but it also does the cropping
            # so not sure what's going on there
            cv2.imwrite(pic_name, img[y-padding+1:y+h+padding, x-padding+1:x+w+padding])
        except:
            pass

crop_faces()
